In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
# Load environment variables
load_dotenv()

# Initialize LLM and memory
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Information_Extractor"
LANGCHAIN_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENAI_API_KEY = os.getenv("OPEN_API")


In [11]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
)

In [13]:
def search_duckduckgo(query: str):
    """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

def multiply(a:int,b:int) -> int:
    """
    Multiply a and b
    """
    return a/ b

def add(a:int,b:int) -> int:
    """
    Adds a and b
    """
    return a + b

In [22]:
maths_agent = create_react_agent(
    model = llm,
    tools = [add, multiply],
    name = "maths_expert", 
    prompt="For maths related task use. Always use only one tool at a time."
    )

research_agent = create_react_agent(
    model=llm,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="for research related task with access to web search return something if possible. Do not do maths."
)


In [23]:
workflow = create_supervisor(
    [research_agent, maths_agent],
    model=llm,
    prompt=(
        "As a team supervisor manage a research expert and maths expert."
        "For events use, research_agent"
        "For math problem, use maths_agent"
    )
)

In [24]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
        "role": "user",
        "content": "what is quantum computing"
        }
    ]
})

In [25]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is quantum computing
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_aFGzrYVo8YHhFHdTgn26kfs8)
 Call ID: call_aFGzrYVo8YHhFHdTgn26kfs8
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Quantum computing is a rapidly-emerging technology that utilizes the principles of quantum mechanics to perform calculations at speeds unattainable by classical computers. It leverages unique quantum phenomena such as superposition and entanglement to process information in fundamentally different ways.

Key concepts include:

- **Qubits**: The basic unit of quantum information, analogous to classical bit

In [26]:
result = app.invoke({
    "messages": [
        {
        "role": "user",
        "content": "what is the weather in delhi. Multiply it by 2 and add 5."
        }
    ]
})

In [27]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is the weather in delhi. Multiply it by 2 and add 5.
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_ZEURIIMF9p7EuvRzOdJeZpQu)
 Call ID: call_ZEURIIMF9p7EuvRzOdJeZpQu
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

The current temperature in Delhi is approximately 38 °C. If you need any further information or assistance, feel free to ask!
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (5c0e5613-05a8-4b7e-b3d4-96398b198b8f)
 Call ID: 5c0e5613-05a8-4b7e-b3d4